# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [18]:
import pandas as pd
import re
import numpy as np
from datetime import datetime
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

s = pd.Series(doc)
s.head(10)

df = pd.DataFrame(s)
df['Date'] = ""
df.columns = ['Text', 'Date']
df.head(5)


month_num = {'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 
             'Apr': 'April', 'May': 'May', 'Jun': 'June',
             'Jul': 'July', 'Aug': 'August', 'Sep': 'September', 
             'Oct': 'October', 'Nov': 'November', 'Dec': 'December'}

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

,Text,Date
0,03/25/93 Total time of visit (in minutes):\n,
1,6/18/85 Primary Care Doctor:\n,
2,sshe plans to move as of 7/8/71 In-Home Servic...,
3,7 on 9/27/75 Audit C Score Current:\n,
4,2/6/96 sleep studyPain Treatment Pain Level (N...,
5,.Per 7/06/79 Movement D/O note:\n,
6,"4, 5/18/78 Patient's thoughts about current su...",
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...,
8,3/7/86 SOS-10 Total Score:\n,
9,(4/10/71)Score-1Audit C Score Current:\n,


In [20]:
month_names = {'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 
             'Apr': 'April', 'May': 'May', 'Jun': 'June',
             'Jul': 'July', 'Aug': 'August', 'Sep': 'September', 
             'Oct': 'October', 'Nov': 'November', 'Dec': 'December'}


In [21]:
def get_mmddyyyy(raw_date):
    format_orig = '%m/%d/%Y'
    regex = "\d{1,2}[\/-]\d{1,2}[\/-]\d{4}"
    tmp = re.findall(regex, raw_date)
    
    if tmp:
        datetime_str = datetime.strptime(tmp[0], format_orig)
        return datetime_str.strftime('%Y/%m/%d')
    return ""

df['Date'] = df['Text'].apply(get_mmddyyyy); # More compact syntax

# Saving preprocessed date
mask = (df['Date'] != "")
df1 = df[mask]
del df1['Text']
df1.head(10)

# Removing preprocessed date from df
mask = (df['Date'] == "")
df = df[mask]
df.head(20)

,Date
14,1990/05/24
15,2011/01/25
17,1976/10/13
24,1984/07/25
30,1985/03/31
34,2012/05/12
38,1986/07/27
43,2002/04/13
45,1998/02/15
55,1987/10/11


,Text,Date
0,03/25/93 Total time of visit (in minutes):\n,
1,6/18/85 Primary Care Doctor:\n,
2,sshe plans to move as of 7/8/71 In-Home Servic...,
3,7 on 9/27/75 Audit C Score Current:\n,
4,2/6/96 sleep studyPain Treatment Pain Level (N...,
5,.Per 7/06/79 Movement D/O note:\n,
6,"4, 5/18/78 Patient's thoughts about current su...",
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...,
8,3/7/86 SOS-10 Total Score:\n,
9,(4/10/71)Score-1Audit C Score Current:\n,


In [22]:
def get_mmddyy(raw_date):
    format_orig = '%m/%d/%Y'
    regex = "\d{1,2}[\/]\d{1,2}[\/]\d{2}"
    match = re.findall(regex, raw_date)
    
    if match:
        year_fix = "19" + match[0][-2:]
        tmp = match[0][0:-2] + year_fix
        try:
            datetime_str = datetime.strptime(tmp, format_orig)
            return datetime_str.strftime('%Y/%m/%d')
        except:
            datetime_str = datetime.strptime(tmp, '%d/%m/%Y')
            return datetime_str.strftime('%Y/%m/%d')
    else:
        return ""
        

df['Date'] = df['Text'].apply(get_mmddyy); # More compact syntax

# Saving preprocessed date
mask = (df['Date'] != "")
df2 = df[mask]
del df2['Text']
df2.head(10)

# Removing preprocessed date from df
mask = (df['Date'] == "")
df = df[mask]
df.head(10)

,Date
0,1993/03/25
1,1985/06/18
2,1971/07/08
3,1975/09/27
4,1996/02/06
5,1979/07/06
6,1978/05/18
7,1989/10/24
8,1986/03/07
9,1971/04/10


,Text,Date
25,4-13-82 Other Child Mental Health Outcomes Sca...,
39,1-14-81 Communication with referring physician...,
40,7-29-75 CPT Code: 90801 - Psychiatric Diagnosi...,
71,4-13-89 Communication with referring physician...,
125,s The patient is a 44 year old married Caucasi...,
126,.10 Sep 2004 - Intake at EEC for IOP but did n...,
127,see above and APS eval of 26 May 1982 Social H...,
128,Tbooked for intake appointment at Sierra Vista...,
129,06 May 1972 SOS-10 Total Score:\n,
130,25 Oct 1987 Total time of visit (in minutes):\n,


In [23]:
def get_mmddyy_barsep(raw_date):
    format_orig = '%m-%d-%Y'
    regex = "\d{1,2}[\-]\d{1,2}[\-]\d{2}"
    match = re.findall(regex, raw_date)
    
    if match:
        year_fix = "19" + match[0][-2:]
        tmp = match[0][0:-2] + year_fix
        try:
            datetime_str = datetime.strptime(tmp, format_orig)
            return datetime_str.strftime('%Y/%m/%d')
        except:
            datetime_str = datetime.strptime(tmp, '%d-%m-%Y')
            return datetime_str.strftime('%Y/%m/%d')
    else:
        return ""
        

df['Date'] = df['Text'].apply(get_mmddyy_barsep); # More compact syntax

# Saving preprocessed date
mask = (df['Date'] != "")
df3 = df[mask]
del df3['Text']
df3.head(10)

# Removing preprocessed date from df
mask = (df['Date'] == "")
df = df[mask]
df.head(10)

4-13-1982
1-14-1981
7-29-1975
4-13-1989


,Date
25,1982/04/13
39,1981/01/14
40,1975/07/29
71,1989/04/13


,Text,Date
125,s The patient is a 44 year old married Caucasi...,
126,.10 Sep 2004 - Intake at EEC for IOP but did n...,
127,see above and APS eval of 26 May 1982 Social H...,
128,Tbooked for intake appointment at Sierra Vista...,
129,06 May 1972 SOS-10 Total Score:\n,
130,25 Oct 1987 Total time of visit (in minutes):\n,
131,14 Oct 1996 SOS-10 Total Score:\n,
132,30 Nov 2007 CPT Code: 90801 - Psychiatric Diag...,
133,h missed intake office visit on 28 June 1994 a...,
134,14 Jan 1981 SOS-10 Total Score:\n,


In [24]:
def get_dd_month_yyyy(raw_date):
    format_orig = '%d %b %Y'
    regex = "\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{2,4}"
    match = re.findall(regex, raw_date)
    
    if match:
        try:
            datetime_str = datetime.strptime(match[0], format_orig)
        except:
            datetime_str = datetime.strptime(match[0], '%d %B %Y')

        return datetime_str.strftime('%Y/%m/%d')
        
    else:
        return ""
        

df['Date'] = df['Text'].apply(get_dd_month_yyyy); # More compact syntax

# Saving preprocessed date
mask = (df['Date'] != "")
df4 = df[mask]
del df4['Text']
df4.head(10)

# Removing preprocessed date from df
mask = (df['Date'] == "")
df = df[mask]
df.head(10)

,Date
125,2001/01/24
126,2004/09/10
127,1982/05/26
128,2002/06/28
129,1972/05/06
130,1987/10/25
131,1996/10/14
132,2007/11/30
133,1994/06/28
134,1981/01/14


,Text,Date
194,"April 11, 1990 CPT Code: 90791: No medical ser...",
195,"MRI May 30, 2001 empty sella but no problems w...",
196,".Feb 18, 1994: made a phone call to Mom and Mo...",
197,"Brother died February 18, 1981 Parental/Caregi...",
198,none; but currently has appt with new HJH PCP ...,
199,".Came back to US on Jan 24 1986, saw Dr. Quack...",
200,"July 26, 1978 Total time of visit (in minutes):\n",
201,father was depressed inpatient at DFC December...,
202,"May 15, 1989 SOS-10 Total Score:\n",
203,"September 06, 1995 Total time of visit (in min...",


5 - Preprocessing  (es. April 11, 1990 or Feb 18, 1981 or February 18 1981)

In [25]:
def get_dd_month_yyyy(raw_date):
    format_orig = '%b %d %Y'
    regex = "(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.? \d{1,2}.? \d{2,4}"
    match = re.findall(regex, raw_date)
    
    if match:
        match[0] = re.sub('[,.]', '', match[0])
        try:
            datetime_str = datetime.strptime(match[0], format_orig)
        except:
            datetime_str = datetime.strptime(match[0], '%B %d %Y')

        return datetime_str.strftime('%Y/%m/%d')     
    else:
        return ""
        
df['Date'] = df['Text'].apply(get_dd_month_yyyy); # More compact syntax

# Saving preprocessed date
mask = (df['Date'] != "")
df5 = df[mask]
del df5['Text']
df5.head(10)

# Removing preprocessed date from df
mask = (df['Date'] == "")
df = df[mask]
df.head(10)

,Date
194,1990/04/11
195,2001/05/30
196,1994/02/18
197,1981/02/18
198,2013/10/11
199,1986/01/24
200,1978/07/26
201,1999/12/23
202,1989/05/15
203,1995/09/06


,Text,Date
228,"s 20 yo M carries dx of BPAD, presents for psy...",
229,t Allergies Sulfa (Sulfonamide Antibiotics) - ...,
230,B/R Walnut Ridge. Raised with sister and paren...,
231,50 yo DWF with a history of alcohol use disord...,
232,")HTN, hypercholesterolemia, DM, sleep apnea,, ...",
233,"Dr. Gloria English, who conducted an initial c...",
234,. Patient presents with GAD flareup in Jul 200...,
235,pOct 2015 - Admitted to Gray Clinic for depres...,
236,Pt does not use marijuana currently. Her last ...,
237,"safter evicted in February 1976, hospitalized ...",


6 - Parsing Month-Years dates (es. April 2005) checking also for misstype errors

In [26]:
def get_month_yyyy(raw_date):
    format_orig = '%d %B %Y'
    regex = "(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.? \d{2,4}.?"
    match = re.findall(regex, raw_date)
    
    if match:
        match[0] = re.sub('[;,.():\"-]', '', match[0]).rstrip()
        
        # Fixing months mistypes
        date = match[0].split(' ')
        day = "1"
        month = date[0]
        year = date[1]
        
        for idx in month_names:
            if idx in match[0]:
                month = month_names[idx]

        match[0] = day + " " + month + " " + year
        datetime_str = datetime.strptime(match[0], format_orig)
        
        return datetime_str.strftime('%Y/%m/%d')
    else:
        return ""
        
df['Date'] = df['Text'].apply(get_month_yyyy); # More compact syntax

# Saving preprocessed date
mask = (df['Date'] != "")
df6 = df[mask]
del df6['Text']
df6.head(10)

# Removing preprocessed date from df
mask = (df['Date'] == "")
df = df[mask]
df.head(10)

,Date
228,1985/09/01
229,2011/06/01
230,1986/05/01
231,2016/05/01
232,1977/07/01
233,1990/07/01
234,2003/07/01
235,2015/10/01
236,1995/05/01
237,1976/02/01


,Text,Date
343,6/1998 Primary Care Doctor:\n,
344,"s 52 y/o MWM h/o chronic depression, anxiety, ...",
345,10/1973 Hx of Brain Injury: Yes\n,
346,9/2005 Primary Care Doctor:\n,
347,s 03/1980 Positive PPD: treated with INH for 6...,
348,12/2005 Family Psych History: Family History o...,
349,5/1987 Primary Care Doctor:\n,
350,5/2004 Primary Care Doctor:\n,
351,A pleasant 28 yo woman with no formal psychiat...,
352,See initial PROMPTCARE evaluation from 3/1986 ...,


In [27]:
def get_mm_yyyy(raw_date):
    format_orig = '%m/%Y'
    regex = "\d{1,2}\/\d{2,4}"
    match = re.findall(regex, raw_date)
    
    if match:
        match[0] = re.sub('[;,.():\"-]', '', match[0]).rstrip()
        datetime_str = datetime.strptime(match[0], format_orig)
        return datetime_str.strftime('%Y/%m/%d')
    else:
        return ""
        
df['Date'] = df['Text'].apply(get_mm_yyyy); # More compact syntax

# Saving preprocessed date
mask = (df['Date'] != "")
df7 = df[mask]
del df7['Text']
df7.head(10)

# Removing preprocessed date from df
mask = (df['Date'] == "")
df = df[mask]
df.head(10)

,Date
343,1998/06/01
344,2005/06/01
345,1973/10/01
346,2005/09/01
347,1980/03/01
348,2005/12/01
349,1987/05/01
350,2004/05/01
351,1974/08/01
352,1986/03/01


,Text,Date
455,sHemmorage caused by probe in 1984 Medical His...,
456,sHas been at MYH since his treaters in NE reti...,
457,Pt joined Army reserves in 2001 and has 3 year...,
458,one sister from whom he is estranged due to he...,
459,sSince 1998. Prior medication trials (includin...,
460,1 Ex-smoker : quit 2012\n,
461,". Age 16, 1991, frontal impact. out for two we...",
462,sLexapro (1988-now): Good response (anxiety)\n,
463,s 25 year old engaged to be married Optic...,
464,8Complications from brain hemmorage in 2016 Ax...,


In [28]:
def get_yyyy(raw_date):
    format_orig = '%Y'
    regex = "\d{4}"
    match = re.findall(regex, raw_date)
    
    if match:
        match[0] = re.sub('[;,.():\"-]', '', match[0]).rstrip()
        datetime_str = datetime.strptime(match[0], format_orig)
        return datetime_str.strftime('%Y/%m/%d')
    else:
        return ""
        
df['Date'] = df['Text'].apply(get_yyyy); # More compact syntax

# Saving preprocessed date
mask = (df['Date'] != "")
df8 = df[mask]
del df8['Text']
df8.head(10)

# Removing preprocessed date from df
mask = (df['Date'] == "")
df = df[mask]
df.head(10)

,Date
455,1984/01/01
456,2000/01/01
457,2001/01/01
458,1982/01/01
459,1998/01/01
460,2012/01/01
461,1991/01/01
462,1988/01/01
463,2014/01/01
464,2016/01/01


45

,Text,Date


In [29]:
format_orig = '%Y'
regex = "\d{4}"
for row in df.itertuples():
    match = re.findall(regex, row.Text)
    if match:
        #print(row.Text)
        print(match[0])
        
        match[0] = re.sub('[;,.():\"-]', '', match[0]).rstrip()
        
        # Fixing months mistypes
        #date = match[0].split(' ')
        #day = "1"
        #month = date[0]
        #year = date[1]
        
        datetime_str = datetime.strptime(match[0], format_orig)
        
        print(datetime_str.strftime('%Y/%m/%d'))

In [32]:
df_result = df1.copy()
df_result = df_result.append(df2)
df_result = df_result.append(df3)
df_result = df_result.append(df4)
df_result = df_result.append(df5)
df_result = df_result.append(df6)
df_result = df_result.append(df7)
df_result = df_result.append(df8)
df_result.head(5)

,Date
14,1990/05/24
15,2011/01/25
17,1976/10/13
24,1984/07/25
30,1985/03/31


In [36]:
len(df_result)
answer = df_result.sort_values(by=['Date'])
answer.reset_index(inplace=True)
del answer['Date']

500

,Date
14,1990/05/24
15,2011/01/25
17,1976/10/13
24,1984/07/25
30,1985/03/31


,index
0,9
1,84
2,2
3,53
4,28
5,474
6,153
7,13
8,129
9,98


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     415
17     335
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    480
477    286
478    244
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64

In [39]:
def date_sorter():
    ans = pd.Series(answer['index'])
        
    return ans

In [38]:
ans = pd.Series(answer['index'])
ans.head()

0     9
1    84
2     2
3    53
4    28
Name: index, dtype: int64